In [1]:
import glob, json, os
import numpy as np

In [2]:
def readJson(path):
    f = open(path)
    data = json.load(f)
    f.close()
    return data

# (q0,q1,q2,q3) = (W, X, Y, Z)
def quaternion_to_rotation_matrix(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    q0 = Q[0]
    q1 = Q[1]
    q2 = Q[2]
    q3 = Q[3]
     
    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)
     
    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)
     
    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1
     
    # 3x3 rotation matrix
    rot_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
                            
    return rot_matrix


In [3]:
def convertTransformMatrix(translation, quaternion):
    
    transform = np.identity(4)

    # T: (X, Y, Z)
    translation = np.array(translation)
    transform[0:3, 3] = translation
    
    # Q: (W, X, Y, Z)
    rotation = quaternion_to_rotation_matrix(quaternion)
    transform[0:3, 0:3] = rotation

    return transform.reshape((1,16))

def convertSceneTemplate(data_paths):
    scene_template = dict()
    
    scene_template['camera_track_map'] = dict()
    scene_template['camera_track_map']['images'] = dict()
    image_path = dict()
    image_path['file_paths'] = dict()
    for data_path in data_paths:
        data = readJson(data_path)
    
        size = [data['metadata']['resolution'][0], data['metadata']['resolution'][1]]
        focal = [data['camera']['K'][0][0]*size[0],
                 -data['camera']['K'][1][1]*size[1]]
        ppt = [-data['camera']['K'][0][2]*size[0],
               -data['camera']['K'][1][2]*size[1]]
    
        for i in range(data['metadata']['num_frames']):
            if 'camera_00_metadata' in data_path:
                filename = '00_' + str(i).zfill(5)
                image_path['file_paths'][filename] = 'images/'+filename+'.png'
            elif 'camera_01_metadata' in data_path:
                filename = '01_' + str(i).zfill(5)
                image_path['file_paths'][filename] = 'images/'+filename+'.png'
            elif 'camera_02_metadata' in data_path:
                filename = '02_' + str(i).zfill(5)
                image_path['file_paths'][filename] = 'images/'+filename+'.png'
            else:
                raise Exception('Camera not supported!')
    
            metadata = dict()
            metadata['flg'] = 2
            metadata['size'] = size
            metadata['camera'] = {'intrinsic': {'focal':focal, 'ppt': ppt},
                                  'extrinsic': convertTransformMatrix(data['camera']['positions'][i], data['camera']['quaternions'][i]).tolist()}

            scene_template['camera_track_map']['images'][filename] = metadata
    
    scene_template['image_path'] = image_path
    scene_template['bbox'] = {'transform': [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0]}
        
    return scene_template

In [4]:
metadata_paths = glob.glob('/home/leyinghu/Documents/data/SMPL/smplx_blender_addon_300_20220623/1000/SMPLX-male_CircularCameraPathAnimator_white_nolights/camera_*_metadata.json')
metadata_paths.sort()
assert len(metadata_paths) == 3

sfm_scene_path = '/home/leyinghu/Documents/ml-neilf/SMPLX-male-1000/inputs/sfm_scene.json'

In [5]:
sfm_scene_template = convertSceneTemplate(metadata_paths)

with open(sfm_scene_path, 'w') as f:
    json.dump(sfm_scene_template, f, indent=2)